# Project Notebook

### Importing Libraries

In [893]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from openpyxl import load_workbook

## 1. Import Datasets

In [894]:
# main data set
data_main = pd.read_csv('data/cancer_reg.csv', encoding = "ISO-8859-1")

# import data of manufactoring share
wb = load_workbook('data/industry_county_data_overview.xlsx', data_only=True)
ws = wb['new_overview']
ws_data = ws.values
# Get the first line in file as a header line
columns = next(ws_data)[0:]
# Create a DataFrame based on the second and subsequent lines of data
data_industry = pd.DataFrame(ws_data, columns=columns)

# import data of UV exposure
data_uv = pd.read_csv('data/uv-county.csv', encoding = "ISO-8859-1")

# import data with rural/urban classification
data_county_class = pd.read_excel('data/PctUrbanRural_County.xls', sheet_name='Pct urban by county')


In [895]:
data_industry = data_industry[['county_state', 'Industry Share of County GDP (Manufacturing)', 'Industry Share of County GDP (Mining, Oil, Gas)']]
data_industry

,county_state,Industry Share of County GDP (Manufacturing),"Industry Share of County GDP (Mining, Oil, Gas)"
0,Autauga_AL,0.15095,0.019244
1,Baldwin_AL,0.068274,0.002761
2,Barbour_AL,0.261618,0.088787
3,Bibb_AL,0.119731,0.030386
4,Blount_AL,0.12457,0.004438
...,...,...,...
3112,Sweetwater_WY,0.110247,0.433276
3113,Teton_WY,0.009897,0.003029
3114,Uinta_WY,0.041912,0.102798
3115,Washakie_WY,0.153798,0.048576


## 2. Data Cleaning
## 2.1 Harmonize county names

### Lower case for all column names

In [896]:
data_main.columns=[item.lower() for item in data_main.columns]
data_industry.columns=[item.lower() for item in data_industry.columns]
data_uv.columns=[item.lower() for item in data_uv.columns]
data_county_class.columns=[item.lower() for item in data_county_class]

In [897]:
#data_main.geography.to_csv('county_check.csv')

In [898]:
# Extract county name from Geography
# data_main['county'] = data_main.geography.str.split(' County', expand=True)[0]
def get_countyname(item):
    if ' County' in item:
        return item[:item.find(' County')]
    elif ' Parish' in item:
        return item[:item.find(' Parish')]
    #elif ' city' in item:
    #    return item[:item.find(' city')]
    else:
        return item[:item.find(',')]

data_main['county'] =  data_main.geography.apply(get_countyname)
# Extract state name from Geography
data_main['state'] = data_main.geography.str.split(', ').str[1]

In [899]:
data_main.state

0       Washington
1       Washington
2       Washington
3       Washington
4       Washington
           ...    
3042        Kansas
3043        Kansas
3044        Kansas
3045        Kansas
3046        Kansas
Name: state, Length: 3047, dtype: object

In [900]:
# read state abbreviation lookup table
wb2 = load_workbook('data/state_abbreviations.xlsx')
ws2 = wb2['Sheet1']
ws_data2 = ws2.values
# Get the first line in file as a header line
columns2 = next(ws_data2)[0:]
# Create a DataFrame based on the second and subsequent lines of data
state_abb = pd.DataFrame(ws_data2, columns=columns2)

In [901]:
state_abb.head()

,state,state_short
0,ALABAMA,AL
1,ALASKA,AK
2,AMERICAN SAMOA,AS
3,ARIZONA,AZ
4,ARKANSAS,AR


In [902]:
# capitalize state names in lookup table
state_abb.state = state_abb.state.str.title()
# Correct this for DC
state_abb.state = state_abb.state.str.replace('District Of Columbia','District of Columbia')

### Merge main table with lookup table

In [903]:
# merge
data_main = pd.merge(data_main, state_abb, how='left', on='state')

In [904]:
print(sorted(data_main.state.unique()))
print(sorted(state_abb.state.unique()))

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana

### Create new column which consists of county and state name

In [905]:
# New combined column
data_main['county_state'] = data_main['county'].map(str) + '_' + data_main['state_short'].map(str)
data_main.county_state

0          Kitsap_WA
1        Kittitas_WA
2       Klickitat_WA
3           Lewis_WA
4         Lincoln_WA
            ...     
3042    Ellsworth_KS
3043       Finney_KS
3044         Ford_KS
3045     Franklin_KS
3046        Geary_KS
Name: county_state, Length: 3047, dtype: object

In [906]:
# check for duplicates
data_main.geography[data_main.county_state.duplicated(keep=False) == True]

Series([], Name: geography, dtype: object)

### Harmonize data_industry table

In [907]:
data_industry.head()

,county_state,industry share of county gdp (manufacturing),"industry share of county gdp (mining, oil, gas)"
0,Autauga_AL,0.15095,0.019244
1,Baldwin_AL,0.068274,0.002761
2,Barbour_AL,0.261618,0.088787
3,Bibb_AL,0.119731,0.030386
4,Blount_AL,0.12457,0.004438


In [908]:
# kick out asterisks and (independent city from county)
data_industry.county_state = data_industry.county_state.str.replace(', ','_', regex=False)
data_industry.county_state = data_industry.county_state.str.replace('*','', regex=False)
data_industry.county_state = data_industry.county_state.str.replace('(Independent City)','city', regex=False)
data_industry.county_state = data_industry.county_state.str.replace('(city)','city', regex=False)
# rename columns
data_industry.rename(columns={"industry share of county gdp (manufacturing)": "industry_share_manufacturing", "industry share of county gdp (mining, oil, gas)": "industry_share_mineoilgas"}, inplace=True)

### Harmonize data_uv table

In [909]:
data_uv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3108 entries, 0 to 3107
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   statename    3108 non-null   object
 1   state_abr    3108 non-null   object
 2   county_fips  3108 non-null   int64 
 3   county name  3108 non-null   object
 4   uv_ wh/m_    3108 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 121.5+ KB


In [910]:
# Create new Column county_state
data_uv['county_state'] = data_uv['county name'].map(str) + '_' + data_uv['state_abr'].map(str)
data_uv['county_state']

0          Autauga_AL
1          Baldwin_AL
2          Barbour_AL
3             Bibb_AL
4           Blount_AL
            ...      
3103    Sweetwater_WY
3104         Teton_WY
3105         Uinta_WY
3106      Washakie_WY
3107        Weston_WY
Name: county_state, Length: 3108, dtype: object

In [911]:
# Rename column
data_uv.rename(columns={"uv_ wh/m_": "UV"}, inplace=True)

# dropping unnessessary columns
data_uv = data_uv[['county_state','UV']]
data_uv.head()

,county_state,UV
0,Autauga_AL,4563
1,Baldwin_AL,4492
2,Barbour_AL,4642
3,Bibb_AL,4499
4,Blount_AL,4416


### Harmonize data_county_class table

In [912]:
data_county_class

,state,county,statename,countyname,pop_cou,area_cou,pop_urban,poppct_urban,area_urban,areapct_urban,...,pop_uc,poppct_uc,area_uc,areapct_uc,popden_uc,pop_rural,poppct_rural,area_rural,areapct_rural,popden_rural
0,1,1,Alabama,Autauga,54571,1539582278,31650,58.00,50882539,3.30,...,0,0.00,0,0.00,NaN,22921,42.00,1488699739,96.70,39.9
1,1,3,Alabama,Baldwin,182265,4117521611,105205,57.72,275001368,6.68,...,41556,22.80,111381848,2.71,966.3,77060,42.28,3842520243,93.32,51.9
2,1,5,Alabama,Barbour,27457,2291818968,8844,32.21,17447656,0.76,...,8844,32.21,17447656,0.76,1312.8,18613,67.79,2274371312,99.24,21.2
3,1,7,Alabama,Bibb,22915,1612480789,7252,31.65,18903236,1.17,...,7252,31.65,18903236,1.17,993.6,15663,68.35,1593577553,98.83,25.5
4,1,9,Alabama,Blount,57322,1669961855,5760,10.05,10874700,0.65,...,5374,9.38,9851721,0.59,1412.8,51562,89.95,1659087155,99.35,80.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72,145,Puerto Rico,Vega Baja,59662,118766810,57170,95.82,72771265,61.27,...,0,0.00,0,0.00,NaN,2492,4.18,45995545,38.73,140.3
3217,72,147,Puerto Rico,Vieques,9301,131488427,8230,88.49,13439063,10.22,...,8230,88.49,13439063,10.22,1586.1,1071,11.51,118049364,89.78,23.5
3218,72,149,Puerto Rico,Villalba,26073,92298550,22564,86.54,39316802,42.60,...,0,0.00,0,0.00,NaN,3509,13.46,52981748,57.40,171.5
3219,72,151,Puerto Rico,Yabucoa,37941,143005172,32192,84.85,62090343,43.42,...,0,0.00,0,0.00,NaN,5749,15.15,80914829,56.58,184.0


In [913]:
# drop state column which contains a number
data_county_class.drop('state', axis=1, inplace = True)
# rename the statename column
data_county_class.rename(columns={'statename': 'state'}, inplace=True)

In [914]:
# merge with abbreviation table
data_county_class = pd.merge(data_county_class, state_abb, how='left', on='state')

In [915]:
# Create new Column county_state
data_county_class['county_state'] = data_county_class['countyname'].map(str) + '_' + data_county_class['state_short'].map(str)
data_county_class['county_state']

0         Autauga_AL
1         Baldwin_AL
2         Barbour_AL
3            Bibb_AL
4          Blount_AL
            ...     
3216    Vega Baja_PR
3217      Vieques_PR
3218     Villalba_PR
3219      Yabucoa_PR
3220        Yauco_PR
Name: county_state, Length: 3221, dtype: object

In [916]:
# Drop unnessessary columns
data_county_class = data_county_class[['county_state','poppct_urban', 'poppct_rural']]
data_county_class.head()

,county_state,poppct_urban,poppct_rural
0,Autauga_AL,58.00,42.00
1,Baldwin_AL,57.72,42.28
2,Barbour_AL,32.21,67.79
3,Bibb_AL,31.65,68.35
4,Blount_AL,10.05,89.95


In [917]:
# Look for duplicates
data_county_class[data_county_class.county_state.duplicated(keep=False)]

,county_state,poppct_urban,poppct_rural
1195,Baltimore_MD,93.47,6.53
1216,Baltimore_MD,100.00,0.00
1578,St. Louis_MO,98.86,1.14
1597,St. Louis_MO,100.00,0.00
2829,Bedford_VA,21.57,78.43
2848,Fairfax_VA,98.61,1.39
2852,Franklin_VA,10.77,89.23
2895,Richmond_VA,0.00,100.00
2896,Roanoke_VA,81.46,18.54
2916,Bedford_VA,97.94,2.06


In [918]:
# Manually change the counties that are actually cities
data_county_class.iloc[1216, data_county_class.columns.get_loc('county_state')] = 'Baltimore city_MD'
data_county_class.iloc[1578, data_county_class.columns.get_loc('county_state')] = 'St. Louis city_MO'
data_county_class.iloc[2916, data_county_class.columns.get_loc('county_state')] = 'Bedford city_VA'
data_county_class.iloc[2925, data_county_class.columns.get_loc('county_state')] = 'Fairfax city_VA'
data_county_class.iloc[2927, data_county_class.columns.get_loc('county_state')] = 'Franklin city_VA'
data_county_class.iloc[2945, data_county_class.columns.get_loc('county_state')] = 'Richmond city_VA'
data_county_class.iloc[2946, data_county_class.columns.get_loc('county_state')] = 'Roanoke city_VA'

## 2.2 Data Joining

### Join data_main with data_industry

In [919]:
data_main = pd.merge(data_main, data_industry, how='left', on='county_state')

In [920]:
data_main[(data_main.industry_share_manufacturing.isna() == True)].count

<bound method DataFrame.count of       avganncount  avgdeathsperyear  target_deathrate  incidencerate  \
127   1962.667684               145             193.5     453.549422   
216    309.000000               120             153.3     396.200000   
225    346.000000               156             176.8     399.000000   
228    155.000000                59             172.1     437.000000   
229   1142.000000               407             150.1     373.700000   
233    493.000000               204             155.9     400.500000   
234    115.000000                58             159.2     338.000000   
235    378.000000               157             156.1     400.200000   
240     97.000000                43             181.8     405.700000   
241    518.000000               191             172.5     449.400000   
247    312.000000               136             176.7     420.800000   
249    176.000000               106             223.4     367.300000   
251    272.000000              

### Fill missing values for industry share with mean share of respective state

In [921]:
data_main['industry_share_manufacturing'] = data_main['industry_share_manufacturing'].fillna(data_main.groupby('state')['industry_share_manufacturing'].transform('mean'))
data_main['industry_share_mineoilgas'] = data_main['industry_share_mineoilgas'].fillna(data_main.groupby('state')['industry_share_mineoilgas'].transform('mean'))
data_main.industry_share_manufacturing.value_counts(dropna = False)

0.000000    295
0.142449     42
0.025427      1
0.152833      1
0.145394      1
           ... 
0.017131      1
0.025018      1
0.014692      1
0.004929      1
0.027242      1
Name: industry_share_manufacturing, Length: 2712, dtype: int64

### Join data_main with data_uv

In [922]:
data_main = pd.merge(data_main, data_uv, how='left', on='county_state')
data_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3053 entries, 0 to 3052
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   avganncount                   3053 non-null   float64
 1   avgdeathsperyear              3053 non-null   int64  
 2   target_deathrate              3053 non-null   float64
 3   incidencerate                 3053 non-null   float64
 4   medincome                     3053 non-null   int64  
 5   popest2015                    3053 non-null   int64  
 6   povertypercent                3053 non-null   float64
 7   studypercap                   3053 non-null   float64
 8   binnedinc                     3053 non-null   object 
 9   medianage                     3053 non-null   float64
 10  medianagemale                 3053 non-null   float64
 11  medianagefemale               3053 non-null   float64
 12  geography                     3053 non-null   object 
 13  avg

In [923]:
data_main[(data_main['UV'].isna() == True)].count

<bound method DataFrame.count of       avganncount  avgdeathsperyear  target_deathrate  incidencerate  \
166         799.0               319             150.8          381.0   
254         405.0               165             135.4          317.6   
255          64.0                58             226.6          265.7   
256          46.0                19             240.8          578.2   
257         250.0                60             178.2          718.9   
258         995.0               378             182.2          458.3   
259         117.0                43             159.3          493.6   
260         320.0               130             207.4          539.0   
261          55.0                16             137.6          447.7   
262          51.0                25             221.7          468.0   
263         120.0                39             180.3          566.3   
264         709.0               270             190.3          491.9   
265         190.0              

### Fill missing values for UV with mean share of respective state

In [924]:
data_main['UV'] = data_main['UV'].fillna(data_main.groupby('state')['UV'].transform('mean'))
data_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3053 entries, 0 to 3052
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   avganncount                   3053 non-null   float64
 1   avgdeathsperyear              3053 non-null   int64  
 2   target_deathrate              3053 non-null   float64
 3   incidencerate                 3053 non-null   float64
 4   medincome                     3053 non-null   int64  
 5   popest2015                    3053 non-null   int64  
 6   povertypercent                3053 non-null   float64
 7   studypercap                   3053 non-null   float64
 8   binnedinc                     3053 non-null   object 
 9   medianage                     3053 non-null   float64
 10  medianagemale                 3053 non-null   float64
 11  medianagefemale               3053 non-null   float64
 12  geography                     3053 non-null   object 
 13  avg

In [925]:
data_main[(data_main['UV'].isna() == True)].head()

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctotherrace,pctmarriedhouseholds,birthrate,county,state,state_short,county_state,industry_share_manufacturing,industry_share_mineoilgas,UV
2318,911.0,343,147.6,402.7,51887,196428,18.1,0.000000,"(51046.4, 54545.6]",41.1,...,2.082702,47.978416,6.030934,Hawaii,Hawaii,HI,Hawaii_HI,0.000000,0.000049,NaN
2319,4646.0,1562,135.0,425.2,73985,998714,9.8,708.911660,"(61494.5, 125635]",36.9,...,0.915383,52.713161,6.457474,Honolulu,Hawaii,HI,Honolulu_HI,0.025372,0.001251,NaN
2320,326.0,130,149.4,392.2,58883,71735,12.3,655.189238,"(54545.6, 61494.5]",41.6,...,0.665796,54.706539,4.798814,Kauai,Hawaii,HI,Kauai_HI,0.000000,0.001090,NaN
2321,734.0,247,139.9,418.3,64916,164637,13.1,103.257469,"(61494.5, 125635]",40.0,...,0.870927,50.799618,6.347296,Maui,Hawaii,HI,Maui_HI,0.008457,0.000797,NaN
2719,8.0,3,203.3,201.3,68387,5702,9.9,0.000000,"(61494.5, 125635]",36.8,...,5.295567,42.882250,4.391052,Aleutians West Census Area,Alaska,AK,Aleutians West Census Area_AK,0.419544,0.023742,NaN


It seems like Alaska and Hawaii are missing in the UV table. I'll later decide if I kick both states from the dataset or kick the UV numbers instead.

### Join data_main with data_county_class

In [926]:
data_main = pd.merge(data_main, data_county_class, how='left', on='county_state')
data_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3053 entries, 0 to 3052
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   avganncount                   3053 non-null   float64
 1   avgdeathsperyear              3053 non-null   int64  
 2   target_deathrate              3053 non-null   float64
 3   incidencerate                 3053 non-null   float64
 4   medincome                     3053 non-null   int64  
 5   popest2015                    3053 non-null   int64  
 6   povertypercent                3053 non-null   float64
 7   studypercap                   3053 non-null   float64
 8   binnedinc                     3053 non-null   object 
 9   medianage                     3053 non-null   float64
 10  medianagemale                 3053 non-null   float64
 11  medianagefemale               3053 non-null   float64
 12  geography                     3053 non-null   object 
 13  avg

In [927]:
df_nan = data_main[(data_main['poppct_urban'].isna() == True)]
df_nan = df_nan[['county', 'county_state']]
df_nan

,county,county_state
254,Alexandria city,Alexandria city_VA
255,Bristol city,Bristol city_VA
256,Buena Vista city,Buena Vista city_VA
257,Charlottesville city,Charlottesville city_VA
258,Chesapeake city,Chesapeake city_VA
259,Colonial Heights city,Colonial Heights city_VA
260,Danville city,Danville city_VA
261,Falls Church city,Falls Church city_VA
263,Fredericksburg city,Fredericksburg city_VA
264,Hampton city,Hampton city_VA


The county_class databases misses a lot of cities. The naming is different here compared to data_main.

### Use thefuzz to find some more matches

In [931]:
from thefuzz import fuzz
from thefuzz import process

C:\Users\flanc\.conda\envs\iron\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [947]:
# Let the fuzz analyze the two columns and find matches in the reference list
data_county_class['name_from_df_nan'] = data_county_class['county_state'].apply(lambda x: process.extractOne(x, df_nan['county_state'].to_list(), scorer=fuzz.token_sort_ratio, score_cutoff=70))

In [950]:
# Here are the matches
data_county_class[data_county_class.name_from_df_nan.isna() == False]

,county_state,poppct_urban,poppct_rural,name_from_df_nan
74,Fairbanks North Star_AK,69.13,30.87,"(Fairbanks North Star Borough_AK, 74)"
79,Ketchikan Gateway_AK,76.81,23.19,"(Ketchikan Gateway Borough_AK, 71)"
82,Matanuska-Susitna_AK,49.71,50.29,"(Matanuska-Susitna Borough_AK, 71)"
85,Northwest Arctic_AK,42.54,57.46,"(Northwest Arctic Borough_AK, 83)"
644,LaSalle_IL,69.79,30.21,"(LaSalle_LA, 90)"
1114,Allen_LA,30.58,69.42,"(LaSalle_LA, 78)"
1142,La Salle_LA,26.81,73.19,"(LaSalle_LA, 95)"
1219,Bristol_MA,90.17,9.83,"(Bristol city_VA, 72)"
2664,La Salle_TX,53.65,46.35,"(LaSalle_LA, 76)"
2838,Charles City_VA,0.00,100.00,"(Charlottesville city_VA, 79)"


In [958]:
# Saving the matches to the list
match_list = data_county_class[data_county_class.name_from_df_nan.isna() == False].index.to_list()
# Exclude false matches
exclude = [644,1114,2664,2838,2866,2945,2946]
match_list = [i for j, i in enumerate(match_list) if j not in exclude]
match_list

[74,
 79,
 82,
 85,
 644,
 1114,
 1142,
 1219,
 2664,
 2838,
 2866,
 2915,
 2917,
 2919,
 2920,
 2921,
 2926,
 2928,
 2945,
 2946,
 2950]